# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [1]:
import ee
from utils import GeometryOperations, ImageOperations, StatsOperations, Visualization
from config import *
import pandas as pd
from datetime import datetime

In [2]:
service_account = 'jupyter-gee-project@ee-avs-dse.iam.gserviceaccount.com'
key_path = '../service_account_key.json'

credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials)

print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Class Definitions

In [3]:
def main(protected_area_name, year):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    stats_ops = StatsOperations()
    viz = Visualization()

    # Load protected area
    pa = load_protected_area(protected_area_name)
    pa_geometry = pa.geometry()

    # Process geometry
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)

    # Process imagery
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    
    # Add indices and calculate gradient
    image = img_ops.add_indices_to_image(composite)
    
    # Process each band and collect statistics
    all_stats = []
    bands_to_process = ['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'EVI', 'NDVI']
    
    for band_name in bands_to_process:
        single_band = image.select(band_name)
        gradient = img_ops.get_gradient_magnitude(single_band)

        # Create select pixels that intersect boundary 
        boundary_pixels = geo_ops.get_pixels_boundary(gradient, pa_geometry, scale=500)
        boundary_pixels = boundary_pixels.clip(pa_geometry.buffer(500))
        
        # Get statistics for both boundary and buffer
        boundary_stats = stats_ops.calculate_gradient_statistics(boundary_pixels, name='boundary')
        buffer_stats = stats_ops.calculate_gradient_statistics(boundary_pixels, name='buffer')
        
        # Combine stats with feature information
        row_stats = {
            'WDPA_PID': pa.get('WDPA_PID').getInfo(),
            'ORIG_NAME': pa.get('ORIG_NAME').getInfo(),
            'GIS_AREA': pa.get('GIS_AREA').getInfo(),
            'band_name': band_name,
            'year': year,
            **boundary_stats,
            **buffer_stats
        }
        all_stats.append(row_stats)
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(all_stats)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = f'results_{protected_area_name.replace(" ", "_")}_{year}_{timestamp}.csv'
    df.to_csv(output_file, index=False)
    print(f'Results saved to {output_file}')
    
    # Return visualization of last processed band
    Map = viz.create_map(pa_geometry, gradient, boundary_pixels)
    return Map, df

In [4]:
Map, results_df = main("Serengeti National Park", 2010)
display(results_df.head())
Map

/usr/local/lib/python3.9/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_0/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_0_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.9/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)


Results saved to results_Serengeti_National_Park_2010_20250610_005216.csv


,WDPA_PID,ORIG_NAME,GIS_AREA,band_name,year,boundary_mean,boundary_stddev,boundary_area,buffer_mean,buffer_stddev,buffer_area
0,916,Serengeti National Park,13123.050301,sur_refl_b01,2010,0.090834,0.072736,6.784019e+08,0.090834,0.072736,6.784019e+08
1,916,Serengeti National Park,13123.050301,sur_refl_b02,2010,0.159656,0.111650,6.784019e+08,0.159656,0.111650,6.784019e+08
2,916,Serengeti National Park,13123.050301,sur_refl_b03,2010,0.045173,0.037308,6.784019e+08,0.045173,0.037308,6.784019e+08
3,916,Serengeti National Park,13123.050301,EVI,2010,0.000110,0.000084,6.784019e+08,0.000110,0.000084,6.784019e+08
4,916,Serengeti National Park,13123.050301,NDVI,2010,0.000042,0.000032,6.784019e+08,0.000042,0.000032,6.784019e+08


Map(center=[-2.3332666874595325, 34.78445430188864], controls=(WidgetControl(options=['position', 'transparent…

-add write out for each step, with identifier for each park
-ray to run in parallel in python
-use glance to check usage